In [2]:
from fastapi import FastAPI

In [1]:
!nvidia-smi

Mon Sep 11 09:12:12 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   29C    P0              68W / 300W |      4MiB / 46068MiB |      4%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
embedded_query = instructor_embeddings.embed_query("What was the name mentioned in the conversation?")


In [9]:
len(embedded_query)

1024

In [3]:
embeddings = instructor_embeddings.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)

In [5]:
len(embeddings[0])

1024

In [1]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, StoppingCriteria, StoppingCriteriaList
from langchain import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

2023-09-11 10:38:29.815694: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 10:38:30.702483: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [124]:
class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False




In [41]:
stop_words = ["\n\n"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [38]:
stop_words_ids

[tensor([    1, 29871,    13,    13])]

In [35]:
tokenizer("*.\n\n", return_tensors='pt')

{'input_ids': tensor([[    1, 20611,    13,    13]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [27]:
tokenizer(" As a software engineer, you design, develop, and test software programs for various applications.\n\n", return_tensors='pt')

{'input_ids': tensor([[    1, 29871,  1094,   263,  7047, 22055, 29892,   366,  2874, 29892,
          2693, 29892,   322,  1243,  7047, 11104,   363,  5164,  8324, 29889,
            13,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [125]:
stop_words_ids=[torch.tensor([13, 13])]

In [43]:
stop_words_ids

[tensor([13, 13])]

In [126]:
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops = stop_words_ids)])

In [3]:
## ENG
model_name_en = "meta-llama/Llama-2-7b-chat-hf"

tokenizer_en = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name_en,
                                             local_files_only=True,
                                             # use_fast=False
                                          # use_auth_token=True,
                                         )


In [9]:
model_en = AutoModelForCausalLM.from_pretrained(model_name_en,
                                                local_files_only=True,
                                                device_map='auto',
                                                torch_dtype=torch.float16,
                                                temperature=0.2, # must be strictly positive float
                                                do_sample=True,
                                                # use_auth_token=True,
                                                #  load_in_8bit=True,
                                                #  load_in_4bit=True
                                               )
pipe_en = pipeline("text-generation",
            model=model_en,
            tokenizer= tokenizer_en,
            # return_full_text=True,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            max_new_tokens = 512,
            do_sample=True,
            top_k=30,
            num_return_sequences=1,
            eos_token_id=tokenizer_en.eos_token_id
            )
llm_en = HuggingFacePipeline(pipeline=pipe_en)

template_en = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""


prompt_template_en = PromptTemplate.from_template(
    template_en
)




Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [127]:
generate_kwargs = {
                    "temperature": 0.2,
                    "stopping_criteria": stopping_criteria
                }

In [53]:
pipe_en("How are you?",
        clean_up_tokenization_spaces=True
)

[{'generated_text': "How are you? It's been a while since we last spoke.\nI'm doing well, thanks for asking! It's great to hear from you again. How have you been?\nI've been good, thanks for asking! It's great to hear from you again too. How's life been treating you lately?\nI've been keeping busy with work and other things, but I'm always happy to catch up with you. How about you?\nI'm glad to hear that you're keeping busy! It's always important to stay active and engaged in life. Is there anything new or exciting happening in your world?\nI'm glad to hear that you're doing well! It's always great to hear from you and catch up on what's going on in your life. How about you? Anything new or exciting happening?\nI'm glad to hear that you're doing well! It's always great to hear from you and catch up on what's going on in your life. How about you? Anything new or exciting happening?\nI'm glad to hear that you're doing well! It's always great to hear from you and catch up on what's going 

In [55]:
prompt_en = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

I am Ethan.

Question: What is my name?
Helpful Answer:"""

In [73]:
pipe_en(prompt_en,
        # clean_up_tokenization_spaces=True,
        **generate_kwargs
)

[{'generated_text': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nI am Ethan.\n\nQuestion: What is my name?\nHelpful Answer: Ethan's name is Ethan.\n\n"}]

In [58]:
pipe_en(prompt_en,
        clean_up_tokenization_spaces=True,
        **generate_kwargs
)

[{'generated_text': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nI am Ethan.\n\nQuestion: What is my name?\nHelpful Answer: Ethan's name is Ethan.\n\n"}]

In [128]:
pipe_en(prompt_en,
        clean_up_tokenization_spaces=True,
        **generate_kwargs
)

[{'generated_text': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nI am Ethan.\n\nQuestion: What is my name?\nHelpful Answer: Ethan's name is Ethan.\n\n"}]

In [10]:


model_name = "FlagAlpha/Llama2-Chinese-7b-Chat"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name,
                                          local_files_only=True,
                                          use_fast=False
                                          # use_auth_token=True,
                                         )
model = AutoModelForCausalLM.from_pretrained(model_name,
                                         local_files_only=True,
                                         device_map='auto',
                                         torch_dtype=torch.float16,
                                         temperature=0.2, # must be strictly positive float
                                         do_sample=True,
                                         # use_auth_token=True,
                                        #  load_in_8bit=True,
                                        #  load_in_4bit=True
                                         )
pipe = pipeline("text-generation",
            model=model,
            tokenizer= tokenizer,
            # return_full_text=True,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            max_new_tokens = 512,
            do_sample=True,
            top_k=30,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
            )
llm = HuggingFacePipeline(pipeline=pipe)

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。

  {context}

  问题: {question}
  答案:"""


prompt_template = PromptTemplate.from_template(
    template
)






Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [36]:
tokenizer_en.encode("tools.\n\n")

[1, 8492, 29889, 13, 13]

In [6]:
!pip show transformers

Name: transformers
Version: 4.32.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: sentence-transformers


2

In [129]:
from fastapi import FastAPI
from pydantic import BaseModel, Field
from typing import Optional, List
app = FastAPI()
class Prompt(BaseModel):
    context: str
    question: str
    lang: str
    model: str = Field(default="FlagAlpha/Llama2-Chinese-7b-Chat")
    temperature: float = Field(default=0.2)

@app.post("/question_answering")
async def get_completion(input: Prompt):
    input_dict = input.dict()

    generate_kwargs = {
                    "temperature": input_dict["temperature"],
                    "stopping_criteria": stopping_criteria
                }

    if input_dict["lang"] == "en_us":

        chain_en = LLMChain(
            llm=llm_en,
            prompt=prompt_template_en,
            llm_kwargs=generate_kwargs
        )
        
        ans = chain_en(
            {
                "context" : input_dict["context"],
                "question": input_dict["question"],
            },
            return_only_outputs=True
        )
    else:
        chain = LLMChain(
            llm=llm,
            prompt=prompt_template,
            llm_kwargs=generate_kwargs
        )
        ans = chain(
            {
                "context" : input_dict["context"],
                "question": input_dict["question"],
            },
            return_only_outputs=True,
            
        )
    return {
        "completion": ans["text"]
    }
@app.get("/")
async def root():
    return {
        "message": "this is get"
    }

In [130]:
import nest_asyncio

# Allow for asyncio to work within the Jupyter notebook cell
nest_asyncio.apply()

In [131]:
import uvicorn
uvicorn.run(app=app, host="127.0.0.1", port=8001)

INFO:     Started server process [100062]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)


INFO:     127.0.0.1:17620 - "POST /question_answering HTTP/1.1" 200 OK
INFO:     127.0.0.1:17622 - "POST /question_answering HTTP/1.1" 200 OK
INFO:     127.0.0.1:17624 - "POST /question_answering HTTP/1.1" 200 OK
INFO:     127.0.0.1:17626 - "POST /question_answering HTTP/1.1" 200 OK
INFO:     127.0.0.1:17648 - "POST /question_answering HTTP/1.1" 200 OK
INFO:     127.0.0.1:17650 - "POST /question_answering HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [100062]
